In [1]:
!pip install datasets -qq
!pip install transformers -qq
!pip install accelerate -qq
!pip install fuzzywuzzy -qq
!pip install python-Levenshtein -qq

In [2]:
import random
from typing import Dict, List, Union
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline

In [3]:
def prepare_message_for_llm(text: Union[str, List[str]], categories: Dict[str, str]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    if len(categories) < 2:
        raise RuntimeError(f'The category list is too small! Expected 2 or more categories, got {len(categories)} ones.')
    categories_ = sorted(list(categories.keys()))
    categories_as_string = ', '.join(categories_[:-1]) + ' и ' + categories_[-1]
    if isinstance(text, str):
        prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                 f'списка тем наиболее представлена в следующем тексте. ' \
                 f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                 f'Список тем: {categories_as_string}.\n'
        for cur in categories_:
            prompt += f'Текст: {" ".join(categories[cur].split())}\nВаш ответ: {cur}\n'
        prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
        messages = [
            {
                'role': 'system',
                'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    else:
        messages = []
        for it in text:
            prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                     f'списка тем наиболее представлена в следующем тексте. ' \
                     f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                     f'Список тем: {categories_as_string}.\n'
            for cur in categories_:
                prompt += f'Текст: {" ".join(categories[cur].split())}\nВаш ответ: {cur}\n'
            prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
            messages.append([
                {
                    'role': 'system',
                    'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ])

    return {'message_for_llm': messages}

# Pipeline

## Загрузим модель

In [4]:
llm_pipeline = pipeline(model='Qwen/Qwen2-1.5B-Instruct', device_map='auto', torch_dtype='auto')

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

 ## Загрузим датасет

In [5]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Выделим категории

In [6]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [7]:
print(validation_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


## Выделим случайные примеры

In [8]:
examples_by_categories = dict()
for current_category in list_of_categories:
    examples_by_categories[current_category] = random.choice(
        train_set.filter(lambda it: it['category'] == current_category)['text']
    )
    print(f'Category: {current_category}\n')
    print(f'Random text: {examples_by_categories[current_category]}\n\n')

Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: entertainment

Random text: Большинство храмовых праздников отмечается в рамках годовщины храма, дня рождения центрального божества или любого другого значимого события, связанного с храмом.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: geography

Random text: В области Серенгети находятся национальный парк Серенгети, заповедник Нгоронгоро, заказник Масва в Танзании и национальный парк Маасай Мара в Кении.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

...

Category: science/technology

Random text: Можно также сделать такие сплавы, которые содержат небольшое количество таких неметаллических элементов, как углерод.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: sports

Random text: Разумеется, первоклассный бегун на полмили, способный пробежать менее чем за две минуты, должен обладать изрядной скоростью, но во что бы то ни стало нужно тренировать выносливость.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: travel

Random text: Самый очевидный способ перелёта в первом или бизнес-классе — это раскошелиться за данную привилегию (или же — вариант получше — заставить вашу компанию заплатить за вас).




## Обернем тексты в prompt для llm

In [9]:
validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [10]:
print(validation_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [11]:
print(validation_set['text'][0])

Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.


In [12]:
print(validation_set_for_llm['message_for_llm'][0])

[{'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Большинство храмовых праздников отмечается в рамках годовщины храма, дня рождения центрального божества или любого другого значимого события, связанного с храмом.\nВаш ответ: entertainment\nТекст: В области Серенгети находятся национальный парк Серенгети, заповедник Нгоронгоро, заказник Масва в Танзании и национальный парк Маасай Мара в Кении.\nВаш ответ: geography\nТекст: Когда Лиггинс работал в больнице, в свободное от работы время он начал расследовать случаи преждевременных родов.\nВаш ответ: health\nТекст: Коалиционные и афганские вой

## Сгенерируем ответы

#### Функция для просмотра количества полученных и реальных категорий

In [22]:
from collections import Counter
import pandas as pd


def get_predictions_stat(y_pred, y_true):
    # Получаем категории из предсказанных и истинных меток
    predicted_categories = y_pred
    true_categories = y_true

    # Считаем количество предсказаний для каждой категории
    predictions_count = Counter(predicted_categories)
    true_count = Counter(true_categories)

    # Получаем уникальные категории из предсказанных и истинных меток
    all_categories = set(predicted_categories).union(set(true_categories))
    results = []

    for category in all_categories:
        count = predictions_count.get(category, 0)  # Количество предсказанных для этой категории
        true_count_val = true_count.get(category, 0)  # Количество истинных для этой категории
        correct_count = sum(1 for true, pred in zip(true_categories, predicted_categories) if true == pred and pred == category)
        accuracy = (correct_count / count) * 100 if count > 0 else 0  # Процент правильных предсказаний

        results.append({
            'Category': category,
            'Predicted Count': count,
            'True Count': true_count_val,
            'Correct Count': correct_count,
            'Accuracy (%)': accuracy
        })

    # Преобразуем результаты в DataFrame для удобного отображения
    results_df = pd.DataFrame(results)
    print("All predicted and real categories:")
    print(list(all_categories))
    return results_df

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
def get_predictions(model, dataset, normalize=False, cut_ss=False):
    def process_text(text):
        # Удаляем окончательные буквы 's' и приводим к нижнему регистру
        while text.endswith('s'):
            text = text[:-1]
        return text.lower()

    # Получаем предсказания от модели
    pred = list(map(
        lambda x: model(x, max_new_tokens=10)[0]['generated_text'][-1]['content'],
        tqdm(dataset['message_for_llm'])
    ))
    true = dataset['category']

    # Обработка предсказаний и истинных значений
    if normalize == True:
      print("Normalize by Levenstein distance")
      pred = list(map(
        lambda it: process.extractOne(it, list_of_categories, scorer=fuzz.token_sort_ratio)[0],
        tqdm(pred)
      ))
    elif cut_ss == True:
      print("Normalize by cutting ss")
      pred = list(map(process_text, pred))
      true = list(map(process_text, dataset['category']))
        
    return pred, true

### Результаты бейзлайна

In [29]:
val_pred, val_true = get_predictions(llm_pipeline, validation_set_for_llm, normalize=True)
print(classification_report(y_true=val_true, y_pred=[x for x in val_pred]))
val_stat_df = get_predictions_stat(val_pred, val_true)
val_stat_df

  0%|          | 0/99 [00:00<?, ?it/s]

Normalize by Levenstein distance


  0%|          | 0/99 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

     entertainment       0.50      0.33      0.40         9
         geography       1.00      0.75      0.86         8
            health       1.00      0.64      0.78        11
          politics       0.72      0.93      0.81        14
science/technology       0.83      0.96      0.89        25
            sports       0.48      0.92      0.63        12
            travel       0.70      0.35      0.47        20

          accuracy                           0.72        99
         macro avg       0.75      0.70      0.69        99
      weighted avg       0.75      0.72      0.70        99

All predicted and real categories:
['entertainment', 'health', 'science/technology', 'sports', 'travel', 'politics', 'geography']


,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,entertainment,6,9,3,50.000000
1,health,7,11,7,100.000000
2,science/technology,29,25,24,82.758621
3,sports,23,12,11,47.826087
4,travel,10,20,7,70.000000
5,politics,18,14,13,72.222222
6,geography,6,8,6,100.000000


In [32]:
test_pred, test_true = get_predictions(llm_pipeline, test_set_for_llm, normalize=True)
print(classification_report(y_true=test_true, y_pred=[x for x in test_pred]))
test_stat_df = get_predictions_stat(test_pred, test_true)
test_stat_df

  0%|          | 0/204 [00:00<?, ?it/s]

Normalize by Levenstein distance


  0%|          | 0/204 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

     entertainment       0.64      0.47      0.55        19
         geography       0.90      0.53      0.67        17
            health       0.88      0.68      0.77        22
          politics       0.87      0.90      0.89        30
science/technology       0.69      0.92      0.79        51
            sports       0.58      0.88      0.70        25
            travel       0.92      0.60      0.73        40

          accuracy                           0.75       204
         macro avg       0.78      0.71      0.73       204
      weighted avg       0.78      0.75      0.75       204

All predicted and real categories:
['entertainment', 'health', 'science/technology', 'sports', 'travel', 'politics', 'geography']


,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,entertainment,14,19,9,64.285714
1,health,17,22,15,88.235294
2,science/technology,68,51,47,69.117647
3,sports,38,25,22,57.894737
4,travel,26,40,24,92.307692
5,politics,31,30,27,87.096774
6,geography,10,17,9,90.000000


#### Дальше будем маппить несуществующие категории в ближайшие по косиноснуму расстоянию существующие 

In [33]:
from datasets import load_dataset
import numpy as np

# Загрузка GloVe эмбеддингов
glove_dataset = load_dataset("karmiq/glove", split="train")
glove_model = {item['word']: np.array(item['embeddings']) for item in glove_dataset}

README.md:   0%|          | 0.00/489 [00:00<?, ?B/s]

glove.6B.50d.parquet:   0%|          | 0.00/181M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
def map_predictions(preds, trues, glove_model):
    trues = ['science' if category == 'science/technology' else category for category in trues]
    preds = ['science' if category == 'science/technology' else category for category in preds]
        
    mapped_predictions = []
    true_categories = list(set(trues))
    true_vectors = np.array([glove_model[category] for category in true_categories])

    for pred in preds:
        if pred in true_categories:
            mapped_predictions.append(pred)
        else:
            # Если предсказанная категория не существует, находим ближайшую
            try:
                pred_vector = glove_model[pred].reshape(1, -1)
                # Вычисляем косинусное сходство между вектором предсказанной категории и векторами истинных категорий
                similarities = cosine_similarity(pred_vector, true_vectors)
                # Находим индекс ближайшей категории
                nearest_index = np.argmax(similarities)
                mapped_predictions.append(true_categories[nearest_index])
                print(f"Mapping {pred} to {true_categories[nearest_index]}")
            except KeyError:
                # Если слово не найдено в модели, добавляем None или другую метку
                mapped_predictions.append(None)

    return mapped_predictions, trues

#### Сгенериурем ответы без какой-либо постобработки и посмотрим на них

In [35]:
raw_val_pred, raw_val_true = get_predictions(llm_pipeline, validation_set_for_llm)
print(classification_report(y_true=raw_val_true, y_pred=[x for x in raw_val_pred]))
raw_val_stat_df = get_predictions_stat(raw_val_pred, raw_val_true)
raw_val_stat_df

  0%|          | 0/99 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

              arts       0.00      0.00      0.00         0
     entertainment       0.67      0.44      0.53         9
         geography       0.71      0.62      0.67         8
            health       0.75      0.55      0.63        11
           history       0.00      0.00      0.00         0
          politics       1.00      0.93      0.96        14
science/technology       0.77      0.92      0.84        25
          security       0.00      0.00      0.00         0
             sport       0.00      0.00      0.00         0
            sports       1.00      0.25      0.40        12
            travel       0.62      0.40      0.48        20

          accuracy                           0.63        99
         macro avg       0.50      0.37      0.41        99
      weighted avg       0.78      0.63      0.67        99

All predicted and real categories:
['entertainment', 'history', 'health', 'science/technology', '

,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,entertainment,6,9,4,66.666667
1,history,1,0,0,0.000000
2,health,8,11,6,75.000000
3,science/technology,30,25,23,76.666667
4,sport,16,0,0,0.000000
5,sports,3,12,3,100.000000
6,travel,13,20,8,61.538462
7,security,1,0,0,0.000000
8,politics,13,14,13,100.000000
9,arts,1,0,0,0.000000


In [43]:
raw_test_pred, raw_test_true = get_predictions(llm_pipeline, test_set_for_llm)
print(classification_report(y_true=raw_test_true, y_pred=[x for x in raw_test_pred]))
raw_test_stat_df = get_predictions_stat(raw_test_pred, raw_test_true)
raw_test_stat_df

  0%|          | 0/204 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

               art       0.00      0.00      0.00         0
     entertainment       0.88      0.37      0.52        19
         geography       0.86      0.71      0.77        17
            health       0.94      0.77      0.85        22
          politics       0.86      0.83      0.85        30
science/technology       0.71      0.92      0.80        51
             sport       0.00      0.00      0.00         0
            sports       0.67      0.24      0.35        25
            travel       0.87      0.65      0.74        40
           weather       0.00      0.00      0.00         0

          accuracy                           0.69       204
         macro avg       0.58      0.45      0.49       204
      weighted avg       0.81      0.69      0.72       204

All predicted and real categories:
['entertainment', 'weather', 'health', 'sport', 'science/technology', 'sports', 'travel', 'art', 'politics', 'geography']


,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,entertainment,8,19,7,87.500000
1,weather,1,0,0,0.000000
2,health,18,22,17,94.444444
3,sport,28,0,0,0.000000
4,science/technology,66,51,47,71.212121
5,sports,9,25,6,66.666667
6,travel,30,40,26,86.666667
7,art,1,0,0,0.000000
8,politics,29,30,25,86.206897
9,geography,14,17,12,85.714286


#### И теперь маппим ответы

In [42]:
embedding_mapped_val_pred, embedding_mapped_val_true = map_predictions(raw_val_pred, raw_val_true, glove_model)
print(classification_report(y_true=embedding_mapped_val_true, y_pred=[x for x in embedding_mapped_val_pred]))
embedding_mapped_val_stat_df = get_predictions_stat(embedding_mapped_val_pred, embedding_mapped_val_true)
embedding_mapped_val_stat_df

Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping security to health
Mapping sport to sports
Mapping history to science
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping arts to science
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
               precision    recall  f1-score   support

entertainment       0.67      0.44      0.53         9
    geography       0.71      0.62      0.67         8
       health       0.67      0.55      0.60        11
     politics       1.00      0.93      0.96        14
      science       0.72      0.92      0.81        25
       sports       0.58      0.92      0.71        12
       travel       0.62      0.40      0.48        20

     accuracy                           0.71        99
    macro avg       0.71      0.68      0

,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,entertainment,6,9,4,66.666667
1,health,9,11,6,66.666667
2,science,32,25,23,71.875000
3,sports,19,12,11,57.894737
4,travel,13,20,8,61.538462
5,politics,13,14,13,100.000000
6,geography,7,8,5,71.428571


In [64]:
embedding_mapped_test_pred, embedding_mapped_test_true = map_predictions(raw_test_pred, raw_test_true, glove_model)
print(classification_report(y_true=embedding_mapped_test_true, y_pred=[x for x in embedding_mapped_test_pred]))
embedding_mapped_test_stat_df = get_predictions_stat(embedding_mapped_test_pred, embedding_mapped_test_true)
embedding_mapped_test_stat_df

Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping sport to sports
Mapping art to science
Mapping weather to travel
               precision    recall  f1-score   support

entertainment       0.88      0.37      0.52        19
    geography       0.86      0.71      0.77        17
       health       0.94      0.77      0.85        22
     politics       0.86      0.83      0.85        30
   

,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,entertainment,8,19,7,87.500000
1,health,18,22,17,94.444444
2,science,67,51,47,70.149254
3,sports,37,25,22,59.459459
4,travel,31,40,26,83.870968
5,politics,29,30,25,86.206897
6,geography,14,17,12,85.714286


Видим что с векторным пространством что-то не так, потому что security маппится в health, history в science, и arts тоже в science.
Попробуем "ручной" маппинг по словарю.cience

In [68]:
def dict_map_prediction(category_mapping, predicted_categories):
    result = []
    for prediction in predicted_categories:
        if prediction in category_mapping:
            print(prediction, 'to', category_mapping[prediction])
            result.append(category_mapping[prediction])
        else:
            result.append(prediction)
    return result


category_mapping_dict = {
    'security': 'geography',
    'history': 'geography',
    'arts': 'entertainment',
    'art': 'entertainment',
    'film': 'entertainment',
    'music': 'entertainment',
    'transportation': 'travel',
    'weather': 'geography',
    'weather/weather phenomena': 'geography',
    'the most appropriate theme for this text is "rel': 'geography',
    'theatre': 'entertainment',
    'sport': 'sports'
}

In [69]:
print(raw_val_true[:5])
print(raw_val_pred[:5])

['sports', 'travel', 'geography', 'science/technology', 'geography']
['sport', 'sport', 'health', 'science/technology', 'travel']


In [70]:
dict_mapped_val_pred = dict_map_prediction(category_mapping_dict, raw_val_pred)
print(classification_report(y_true=raw_val_true, y_pred=[x for x in dict_mapped_val_pred]))
dict_mapped_val_stat_df = get_predictions_stat(dict_mapped_val_pred, raw_val_true)
dict_mapped_val_stat_df

sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
security to geography
sport to sports
history to geography
sport to sports
sport to sports
sport to sports
arts to entertainment
sport to sports
sport to sports
sport to sports
sport to sports
                    precision    recall  f1-score   support

     entertainment       0.71      0.56      0.63         9
         geography       0.56      0.62      0.59         8
            health       0.75      0.55      0.63        11
          politics       1.00      0.93      0.96        14
science/technology       0.77      0.92      0.84        25
            sports       0.58      0.92      0.71        12
            travel       0.62      0.40      0.48        20

          accuracy                           0.72        99
         macro avg       0.71      0.70      0.69        99
      weighted avg       0.72      0.72      0.71        99

All predicted an

,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,entertainment,7,9,5,71.428571
1,health,8,11,6,75.000000
2,science/technology,30,25,23,76.666667
3,sports,19,12,11,57.894737
4,travel,13,20,8,61.538462
5,politics,13,14,13,100.000000
6,geography,9,8,5,55.555556


In [71]:
dict_mapped_test_pred = dict_map_prediction(category_mapping_dict, raw_test_pred)
print(classification_report(y_true=raw_test_true, y_pred=[x for x in dict_mapped_test_pred]))
dict_mapped_test_stat_df = get_predictions_stat(dict_mapped_test_pred, raw_test_true)
dict_mapped_test_stat_df

sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
sport to sports
art to entertainment
weather to geography
                    precision    recall  f1-score   support

     entertainment       0.89      0.42      0.57        19
         geography       0.87      0.76      0.81        17
            health       0.94      0.77      0.85        22
          politics       0.86      0.83      0.85        30
science/technology       0.71      0.92      0.80        51
            sports       0.59      0.88      0.71        25
            travel       0.87      0.65      0.74        40

          accuracy          

,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,entertainment,9,19,8,88.888889
1,health,18,22,17,94.444444
2,science/technology,66,51,47,71.212121
3,sports,37,25,22,59.459459
4,travel,30,40,26,86.666667
5,politics,29,30,25,86.206897
6,geography,15,17,13,86.666667


## Разница между dict и embedding маппингом на уровне погрешности, но и тот и тот метод дает precision на 1-2% больше, и macro avg precision также на 1-2% выше